# Retrieval-Augmented Generation Pipeline (Gemini)

This notebook implements the **final RAG pipeline** by integrating:
- Semantic retrieval from a FAISS vector store
- Answer generation using Google Gemini Pro

The system produces **grounded answers** along with retrieved source chunks.


In [ ]:
import os

from dotenv import load_dotenv

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA


In [ ]:
load_dotenv()

print("Environment variables loaded.")


In [ ]:
VECTOR_DB_PATH = "../vectorstore/faiss_index"

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.load_local(
    VECTOR_DB_PATH,
    embeddings,
    allow_dangerous_deserialization=True
)

print("FAISS vector store loaded.")


In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    temperature=0.2
)

print("Gemini LLM initialized.")


In [ ]:
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 4}
)

print("Retriever configured.")


In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True
)

print("RAG pipeline constructed.")


In [ ]:
query = "What is the main topic discussed in this video?"

response = qa_chain(query)

print("Answer:\n")
print(response["result"])


In [ ]:
print("\nRetrieved source chunks:\n")

for i, doc in enumerate(response["source_documents"], start=1):
    print(f"--- Source {i} ---")
    print(doc.page_content[:400])
    print()


In [ ]:
follow_up_query = "Explain the key idea in simple terms."

follow_up_response = qa_chain(follow_up_query)

print("Follow-up Answer:\n")
print(follow_up_response["result"])


In [ ]:
print("RAG pipeline executed successfully.")


## Observations

- The RAG system generates coherent and context-aware answers.
- Retrieved chunks align well with the user query, validating the embedding and chunking strategy.
- Gemini Pro produces concise and grounded responses when provided with retrieved context.

This confirms the effectiveness of the end-to-end RAG pipeline.


## Summary

- Loaded a persisted FAISS vector store
- Retrieved semantically relevant transcript chunks
- Generated grounded answers using Gemini Pro
- Returned source documents for transparency

The RAG pipeline is now **complete and reusable**.

This notebook serves as the **final experimental validation** before
consolidating the system into `main.py`.
